In [1]:
import tkinter
print("Tkinter is working!")


Tkinter is working!


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
from scipy.spatial import distance as dist
import time
import threading
import tkinter as tk

# EAR calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# Global control flag
blink_control_enabled = False

def start_blink_detector():
    global blink_control_enabled
    # Mediapipe setup
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)
    eye_indices = [33, 160, 158, 133, 153, 144]  # Right eye

    cap = cv2.VideoCapture(0)
    closed_frames = 0
    blink_timestamps = []
    last_blink_time = 0
    last_strain_msg = time.time()
    long_eye_closure_start = None

    while True:
        if not blink_control_enabled:
            time.sleep(0.5)
            continue

        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0].landmark
            eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_indices]
            ear = eye_aspect_ratio(eye)

            if ear < 0.23:  # eyes closed
                closed_frames += 1
                if long_eye_closure_start is None:
                    long_eye_closure_start = time.time()
                elif time.time() - long_eye_closure_start >= 1.5:
                    print("[ACTION] Eyes closed too long → Pausing video")
                    pyautogui.press('space')
                    long_eye_closure_start = None
            else:
                if closed_frames >= 3:
                    now = time.time()
                    blink_timestamps = [t for t in blink_timestamps if now - t < 1.2]
                    blink_timestamps.append(now)

                    if len(blink_timestamps) == 1:
                        print("[ACTION] Single blink → Next reel")
                        pyautogui.press('down')
                    elif len(blink_timestamps) == 2:
                        print("[ACTION] Double blink → Previous reel")
                        pyautogui.press('up')
                        blink_timestamps = []  # Reset after double blink
                closed_frames = 0
                long_eye_closure_start = None

            # Show eye care reminder every 10 minutes
            if time.time() - last_strain_msg > 600:
                print("💡 Reminder: Massage your eyes!")
                last_strain_msg = time.time()

            # Draw eyes
            for (x, y) in eye:
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        cv2.imshow("Blink to Scroll", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# Start blink detection in a thread
def start_detection_thread():
    threading.Thread(target=start_blink_detector, daemon=True).start()

# GUI
def toggle_blink_control():
    global blink_control_enabled
    blink_control_enabled = not blink_control_enabled
    toggle_btn.config(text="Turn OFF" if blink_control_enabled else "Turn ON")

root = tk.Tk()
root.title("Blink Reel Controller")
root.geometry("300x150")

title = tk.Label(root, text="👁️ Blink to Scroll Reels", font=("Arial", 14))
title.pack(pady=10)

toggle_btn = tk.Button(root, text="Turn ON", font=("Arial", 12), command=toggle_blink_control)
toggle_btn.pack(pady=10)

exit_btn = tk.Button(root, text="Exit", font=("Arial", 10), command=root.destroy)
exit_btn.pack()

# Start camera logic in background
start_detection_thread()
root.mainloop()


[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION] Double blink → Previous reel
[ACTION] Single blink → Next reel
[ACTION]